In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.7 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

fastai's Layered API

In [3]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [4]:
path = untar_data(URLs.IMDB)
dls = DataBlock(
    blocks=(TextBlock.from_folder(path),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

Transforms

In [5]:
files = get_text_files(path, folders = ['train', 'test'])
txts = L(o.open().read() for o in files[:2000])

In [6]:
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#185) ['xxbos','i','wanted','to','like','this','film',',','yes','its'...]

In [7]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:10]

TensorText([  2,  18, 393,  14,  50,  20,  33,  11, 471, 114])

In [8]:
nums_dec = num.decode(nums[0][:10]); nums_dec

(#10) ['xxbos','i','wanted','to','like','this','film',',','yes','its']

In [9]:
tok.decode(nums_dec)

'xxbos i wanted to like this film , yes its'

In [10]:
tok((txts[0], txts[1]))

((#185) ['xxbos','i','wanted','to','like','this','film',',','yes','its'...],
 (#441) ['xxbos','xxmaj','let',"'s",'face','it',';','some','lame','kid'...])

Writing a transform

In [11]:
def f(x:int): return x+1
tfm = Transform(f)
tfm(2),tfm(2.0)

(3, 2.0)

In [12]:
@Transform
def f(x:int): return x+1
f(2),f(2.0)

(3, 2.0)

In [13]:
class NormalizeMean(Transform):
    def setups(self, items): self.mean = sum(items)/len(items)
    def encodes(self, x): return x-self.mean
    def decodes(self, x): return x+self.mean

In [14]:
tfm = NormalizeMean()
tfm.setup([1,2,3,4,5])
start = 2
y = tfm(start)
z = tfm.decode(y)
tfm.mean,y,z

(3.0, -1.0, 2.0)

Pipeline

In [15]:
tfms = Pipeline([tok, num])
t = tfms(txts[0]); t[:20]

TensorText([   2,   18,  393,   14,   50,   20,   33,   11,  471,  114,   12,    7,  242,   11,    6,  144, 1696, 5680,   29,   18])

In [16]:
tfms.decode(t)[:100]

'xxbos i wanted to like this film , yes its a xxup saw , xxwrep 3 blah ripoff but i like those films '

TfmdLists and Datasets: Transformed Collections

TfmdLists

In [17]:
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize])

In [18]:
t = tls[0]; t[:20]

TensorText([   2,   19,  488,   15,   53,   20,   32,   11,  445,  115,   13,    7,  234,   11,    6,  153, 3359, 6856,   31,   19])

In [19]:
tls.decode(t)[:100]

'xxbos i wanted to like this film , yes its a xxup saw , xxwrep 3 blah ripoff but i like those films '

In [20]:
tls.show(t)

xxbos i wanted to like this film , yes its a xxup saw , xxwrep 3 blah ripoff but i like those films . xxmaj if done well this had all the ingredients of being a good , not brilliant , but good film … xxunk those ingredients had gone off ! xxmaj the acting was terrible , and this was first seen when the captives are introduced with their captor one by one ( hoods taken off ) , the remarks and one liners are just terrible , yes i know , bad writing … .but this is more than that , it was bad writing coupled with bad acting . xxmaj two of the captives had been in a relationship with each other and did not even acknowledge this until a lot further into the film … .. 

 xxmaj sorry , xxmaj i m even wondering why i am bothering to review this movie at all . 

 i will end with xxup plot xxup holes , xxup plot xxup holes & xxup more xxup plot xxup holes ! xxup disappointing !


In [21]:
cut = int(len(files)*0.8)
splits = [list(range(cut)), list(range(cut,len(files)))]
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize],
                splits=splits)

In [22]:
tls.valid[0][:20]

TensorText([    2,     8,   234,    20,    32,   318,     9,     8, 14091,   203,     8, 16052,  3105,    18,     8,  6262,    10,    19,    25,    52])

In [23]:
lbls = files.map(parent_label)
lbls

(#50000) ['neg','neg','neg','neg','neg','neg','neg','neg','neg','neg'...]

In [24]:
cat = Categorize()
cat.setup(lbls)
cat.vocab, cat(lbls[0])

(['neg', 'pos'], TensorCategory(0))

In [25]:
tls_y = TfmdLists(files, [parent_label, Categorize()])
tls_y[0]

TensorCategory(0)

Datasets

In [26]:
x_tfms = [Tokenizer.from_folder(path), Numericalize]
y_tfms = [parent_label, Categorize()]
dsets = Datasets(files, [x_tfms, y_tfms])
x,y = dsets[0]
x[:20],y

(TensorText([   2,   19,  488,   15,   53,   20,   32,   11,  445,  115,   13,    7,  234,   11,    6,  153, 3359, 6856,   31,   19]),
 TensorCategory(0))

In [27]:
x_tfms = [Tokenizer.from_folder(path), Numericalize]
y_tfms = [parent_label, Categorize()]
dsets = Datasets(files, [x_tfms, y_tfms], splits=splits)
x,y = dsets.valid[0]
x[:20],y


(TensorText([    2,     8,   234,    20,    32,   318,     9,     8, 14091,   203,     8, 16052,  3105,    18,     8,  6262,    10,    19,    25,    52]),
 TensorCategory(1))

In [28]:
t = dsets.valid[0]
dsets.decode(t)

('xxbos xxmaj saw this film during the xxmaj mod & xxmaj rockers fest in xxmaj august . i was so inspired and touched . xxmaj harry had an amazing life and one of the best and distinct voices ever recorded . xxmaj for those of you who do n\'t know about xxmaj harry xxmaj nilsson do a little research and you \'ll see that xxmaj harry has probably found his way into your life in one way or another - maybe it was his 70s special " the xxmaj point " or " everybody \'s xxmaj talking " from xxmaj midnight xxmaj cowboy . xxmaj for me it started with " people let me tell you bout my best friend " - the theme song from " the xxmaj courtship of xxmaj eddie \'s xxmaj father . " xxmaj watching this film you can really feel the love and admiration from xxmaj harry \'s true friends and peers . xxmaj do n\'t shed a tear for xxmaj harry - he had a ball … \n\n xxmaj brett',
 'pos')

In [29]:
dls = dsets.dataloaders(bs=64, before_batch=pad_input)

In [30]:
tfms = [[Tokenizer.from_folder(path), Numericalize], [parent_label, Categorize]]
files = get_text_files(path, folders = ['train', 'test'])
splits = GrandparentSplitter(valid_name='test')(files)
dsets = Datasets(files, tfms, splits=splits)
dls = dsets.dataloaders(dl_type=SortedDL, before_batch=pad_input)

In [31]:
path = untar_data(URLs.IMDB)
dls = DataBlock(
    blocks=(TextBlock.from_folder(path),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

Applying the Mid-Level Data API: SiamesePair